In [1]:
# Checks if a file line is a user line
def isUserLine(line):
    lineSplit = line.split('|')
    tabSplit = line.split('\t')
    if len(lineSplit) == 2 and len(tabSplit) == 1:
        #Got a user line pattern
        return True 

    
#---------------------------------------------------------------------------------    
#rmse function for lists
import numpy as np

def rmse(predictions, targets):
    return np.sqrt(((np.array(predictions) - np.array(targets)) ** 2).mean())


print('rmse test1: %s' % (rmse([2,2,2],[2,2,2]) == 0.0))
print('rmse test2: %s' % (rmse([2,2,2],[1,1,1]) == 1.0))
print('rmse test3: %s' % (rmse([2,2,2],[1,1,0]) == np.sqrt(2.0)))

        
#def  

rmse test1: True
rmse test2: True
rmse test3: True


In [2]:
#Read nrRaints rating items lines from file and turn to a handable object 
def readUserData(nrRatings,file):
    #Create data object
    # Called: userData[3][39] 3: item index, 39: is ratingIndex
    # Uses zero-indexing
    nrDataTracked = 4
    userData = [[0 for x in range(0,nrRatings)] for y in range(0,nrDataTracked)]
    
    #Read each line in file
    for i in range(0,nrRatings):
        line = file.readline()
        tabSplit = line.split('\t')
        if len(tabSplit) == 4: #DEFFENSIVE should not be anything else
            #We have a rating-line
            # Save to userData:
            #trackID  
            userData[0][i] = int(tabSplit[0])        
            #rating
            userData[1][i] = int(tabSplit[1])
            #date 
            userData[2][i] = int(tabSplit[2])
            #time 
            userData[3][i] = tabSplit[3].rstrip()
        else:
            print('Error: Rating item line expetcted')
    return userData

# A function for converting CCM index to trackID
# 
# PRE: nodes is the dictionary containing node data 
#      index is the index to be mapped to trackID
def indexToTrackID(index,nodes):
    if 0 <= index and index < len(nodes):
        return int(nodes[index]['name'])
    else: 
        print('Error: index not in Nodes')
        return 0



### Convert json data to graph object

In [3]:
# Import libraries
import json
from pprint import pprint
import networkx as nx
import numpy as np



def createGraphFromJSON(jsonFile):
    with open(jsonFile) as data_file:    
        data = json.load(data_file)

    nodes = data['nodes'] #719 for 10k_users
    links = data['links'] #1000 for 10k_users
    del data #Free space   

    #Create Graph object G from JSON data, G is symmetrical
    G = nx.Graph()

    #index = 0
    for node in nodes:  
        #  Structure ex: {"name" : "20737", "group" : 0.0}
        #print("Item Nmae: %s, json index: %d" % (node["name"],index))
        G.add_node(int(node['name']))
        #index = index + 1

    
    for link in links:
        # Example structure: {"source" : 9,"target" : 0,"value" : 0.9920000000000001}   
        # G.add_edge(3,4,weight=0.8) # specify edge data
        source = indexToTrackID(link['source'],nodes)
        target = indexToTrackID(link['target'],nodes)
        sim = float(link['value'])
        dist = 1 - sim
        
        # weight=w
        G.add_edge(source,target,similarity = sim, distance = dist)

    nx.freeze(G)
    return G


### A few test cases for above scripts and functions

In [4]:
#JSON file to handle 
#file = '10k_users_similarities.json' #uses the 1% user data case
file = 'data/10k_artists_similarities_v1.json'
G = createGraphFromJSON(file)

# Statistics tests of the graph creatin
print('Correct nr of nodes: %s' % (G.number_of_nodes() == 23267))
print('Correct nr of links: %s' % (G.number_of_edges() == 146525))

jsonFile = 'data/10k_artists_similarities_v1.json'
G = createGraphFromJSON(jsonFile)

print('Node test 2: %s' % (len(G.nodes())==23267))
print('Link test 2: %s' % (len(G.edges())==146525))



#NOTE: Test are only vialbe for initial 10k_users dataset
#print('Test1: %s' % (indexToTrackID(0,nodes) == 20737))
#print('Test2: %s' % (indexToTrackID(718,nodes) == 19053))

# NOTE: only viable for 10k_artists dataset 
#print('Test1: %s' % (indexToTrackID(0,nodes) == 293853))
#print('Test2: %s' % (indexToTrackID(23267-1,nodes) == 110254))
#len(nodes)
#TODO: How to get nodes now???


Correct nr of nodes: True
Correct nr of links: True
Node test 2: True
Link test 2: True


In [78]:
#maxSteps = 2 #Maximum nr of steps on the graph for KNN
#k = 3 #Maximum nr of neighbours to consider for prediction

defaultValue = 0
predictItemMissing = 'predItemMissing'
noHistoryMatch = 'noMatchInHistoryItem'
ok = 'predictionMade'
defaultPrediction = 0

k = 2



# Prediction for one user, based on ratings in history
def predictOnGraph(graph,predictData,nrPredictions,history,kValue):

    predictItemErrors = 0    
    predictions = [0] * nrPredictions
    
    #make prediction for each item in predictData
    for i in range(0,nrPredictions):
        
        trackID = predictData[0][i]
        # Do not use rating
        # rating = predictData[1][i]
        date = predictData[2][i]
        time = predictData[3][i]
        
        if trackID in graph:
            # trackID is in the graph
            #Turn history into similarity values relative to track:trackID
            simObject = historyToSimilarities(graph,trackID,history)

            #Find KNN from similarity object
            prediction,status = knnOnSimilarities(simObject,kValue)
            
            predictions[i] = prediction
            
            #TODO: handle statuses: noHistoryMatch
            
        else:
            ##TODO: Handle THIS: 
            #trackID not in Graph
            predictItemErrors = predictItemErrors + 1
            predictions[i] = defaultPrediction
            
            #return(defaultValue,predictItemMissing)
        
        
        
    return predictions,predictItemErrors

# If k > nr available data points, all datapoints are used
def knnOnSimilarities(simObject,k):
    if k < 1:
        print('ERROR: invalid k-value: k < 1')
        return None
    
    if sum(simObject[0]) == 0: 
        # NO similarities found
        return(0,noHistoryMatch)
    
    l = len(simObject[0])
    if k > l:
        #We do not have k objects to work with 
        k = l #Use all items available

    similarityList = [0] * k #the weights
    valueList = [0] * k #Old ratings
    
    #TODO: repeat k times    
    #in k steps, find largest similarities
    for i in range(0,k):
        
        largestSim = max(simObject[0]) #Get largest similarity value
        
        if largestSim > 0:
            #We have actually got a value to save
            index = simObject[0].index(largestSim)
            
            #Save values and remove old max
            similarityList[i] = largestSim
            valueList[i] = simObject[1][index] 
            simObject[0][index] = 0
            
        else:
            #No more similarities to work with
            break
            
    #The lists are processed
    prediction = np.average(valueList,weights=similarityList)
    
    return(prediction,ok) 
    #For calculatiing weighted aberage:
    # np.average(list1,weights=list2)
    
    

#jsonFile = 'data/10k_artists_similarities.json'
#G = createGraphFromJSON(jsonFile)

# HELP FUNCTION FOR STATISTICS
def checkTrackExistence(G,trackList):
    hit = 0
    miss = 0
    
    for track in trackList:
        if track in G:
            hit = hit + 1
        else:
            miss = miss + 1
    
    return (hit,miss)

# Transform history data into similarity-data, based on sim. to track 
# track is guaranteed to exist in graph by this stage, but not items in the graph 
def historyToSimilarities(graph,track,history):
    # Called: history[3][39] 3: item index, 39: is ratingIndex
    #print(history)
    nrObjects = len(history[0])
    nrDataTracked = 2
    
    simObject = [[0 for x in range(0,nrObjects)] for y in range(0,nrDataTracked)]
    
    #trackID :  userData[0][i]         
    #rating  :  userData[1][i] 
    #date    :  userData[2][i] 
    #time    :  userData[3][i] 
    
    for i in range(0,nrObjects):        
        historyItemID = history[0][i]
        if historyItemID in graph:
            # history exists
            simVal = similarity(graph,track,historyItemID)
            simObject[0][i] = simVal #Similarity value
            simObject[1][i] = history[1][i] #rating
        else:
            # History item is not in graph 
            # Similarity = 0
            simObject[0][i] = 0 #Similarity value
            simObject[1][i] = 0
            
    return simObject



In [79]:
#np.average(range(1,11), weights=range(10,0,-1))

list1 = [2,3,3,1]
list2 = [0.9,0.4,0.1,0]

np.average(list1,weights=list2)

list3 = [0] * 9
list3

L = range(3,4)

for i in range(4,4):
    print('Hej')

#list1.index(3)
 
sum([4,3,0])
    

7

In [96]:
# 218079 in G TRUE, from validation userID: 3
# TRAIN data: 5980 in G, from user 3
# 173094
#file = 'data/10k_artists_similarities_v1.json'
jsonFile = 'data/10k_artists_similarities.json'
G = createGraphFromJSON(jsonFile)

p1 = 5980
p2 = 173094
p3 = 218079
p4 = 263935

#nx.astar_path(G,p3,p4,weight = 'distance')
# [218079, 475789, 149530, 431926, 328827, 263935]
#path = [218079, 475789, 149530, 431926, 328827, 263935] NIVE WRONG PATH
#path = [218079, 450065, 579433, 105603, 526173, 263935]
path = [218079, 475789, 458576, 107772, 279203, 263935]
#nx.has_path(G,p3,p4)

manual = G[218079][475789]['similarity']*G[458576][475789]['similarity']*G[458576][107772]['similarity']*G[279203][107772]['similarity']*G[279203][263935]['similarity']

print('Similarity Test: %s' % (similarity(G,p3,p4) == manual))

#194044	90	5112	11:20:00

# Craete dummy data to test on
nrPredictions = 3
predictData = [[0 for x in range(0,nrPredictions)] for y in range(0,4)]

predictData[0][0] = 263935
predictData[1][0] = 50
predictData[2][0] = 1000
predictData[3][0] = '11:20:00'

predictData[0][1] = 45
predictData[1][1] = None
predictData[2][1] = None
predictData[3][1] = None

predictData[0][2] = 339273 #has NO path to history items
predictData[1][2] = 100
predictData[2][2] = 0
predictData[3][2] = '0'



nrHistoryItems = 4
historyData = [[0 for x in range(0,nrHistoryItems)] for y in range(0,5)]
historyData[0][0] = 30963 # 'similarity': 0.35423320495244315
historyData[1][0] = 70
historyData[2][0] = 1000
historyData[3][0] = '11:20:00'
historyData[4][0] = 0.35423320495244315

historyData[0][1] = 207588 #'similarity': 0.20945050558664924
historyData[1][1] = 50
historyData[2][1] = 1000
historyData[3][1] = '11:20:00'
historyData[4][1] = 0.20945050558664924

historyData[0][2] = 328827 # 'similarity': 0.1070148336463973}
historyData[1][2] = 40
historyData[2][2] = 1000
historyData[3][2] = '11:20:00'
historyData[4][2] = 0.1070148336463973

historyData[0][3] = 45 # NOT in graph
historyData[1][3] = 0
historyData[2][3] = None
historyData[3][3] = None
historyData[4][3] = 0





#30963: {'distance': 0.6457667950475569, 'similarity': 0.35423320495244315
#207588: {'distance': 0.7905494944133508, 'similarity': 0.20945050558664924}
#328827: {'distance': 0.8929851663536027, 'similarity': 0.1070148336463973}

kValue = 5

predictions,predictItemErrors = predictOnGraph(G,predictData,nrPredictions,historyData,kValue)

manualPrediction = np.average(historyData[1],weights=historyData[4])
#print(predictions)

print('PredictOnGraph test 1: %s' % (predictions[0] == manualPrediction))

print('PredictOnGraph test 2: %s' % (predictions[1] == defaultPrediction))
print('PredictOnGraph test 3: %s' % (predictions[2] == defaultPrediction))

# OBS: PATH LENGTH == SUM(weights) in the path
#nx.astar_path_length(G,path[0],path[2],weight = 'distance')
# 0.6122942380734302 For 0 - 5
# 0.5052794044270329 For 0 - 4

# 0.2943014588599292  For 0 - 2 
# 0.15839456374793615 For 0 - 1 == Weight 


#G.edges(path[0],data=True)


Similarity Test: True
PredictOnGraph test 1: True
PredictOnGraph test 2: True
PredictOnGraph test 3: True


In [97]:
G[p4]

targetPoint = 263935

#testHistory = 


45 in G


#263935 in G


l2 = [30963,207588,328827]


node = 339273

print(node in G,nx.has_path(G,node,l2[2]))


predictions

predictItemErrors

True False


1

In [11]:
# Returns similarity value obtained from shortest bath between A and B
# 0 if no path exists
# PRE: G is a similarity graph
def similarity(G,A,B):
    if nx.has_path(G,A,B):
        #Return similarity value
        path = nx.astar_path(G,A,B,weight = 'distance')
        steps = len(path) - 1
        similarities = [None] * steps
        
        for i in range(0,steps):
            similarities[i] = G[path[i]][path[i+1]]['similarity']
        
        return aggregateSimilarities(similarities)
    else:
        #No path exists
        return 0

#Aggregate similarities in a fitting way
def aggregateSimilarities(simList):
    ans = 1    
    if(len(simList) < 1):
        #print('ERROR: too few elements in similarity list')
        return 0
    
    for i in range(0,len(simList)):
        ans = ans * simList[i]
        
    return ans
    
    

#G[path[0]][path[1]] #For getting dictionary:
#{'distance': 0.8416054362520639, 'similarity': 0.15839456374793615}

#G[path[2]][path[1]]['similarity'] #For accessing weight values

#obj = nx.all_shortest_paths(G,p3,p4) #Show all shortest path from a node
# [218079, 475789, 458576, 453053, 69990, 263935]

#for o in obj:
#    print(o)

In [12]:
# PRE: Graph succeffully created

#Nr of users to validate on


# Files to use:
#trainFile = 'Webscope_C15/ydata-ymusic-kddcup-2011-track1/trainIdx1.txt'
trainFile = 'data/temp/artistDataset_10000_users.txt'

#validationFile = 'Webscope_C15/ydata-ymusic-kddcup-2011-track1/validationIdx1.txt'
validationFile = 'data/temp/artistValidation_10000_users.txt'


#testFile = 'Webscope_C15/ydata-ymusic-kddcup-2011-track1/testIdx1.txt'
testFile = 'data/temp/artistTest_10000_users.txt'



In [109]:
#def 
maxNrUsers   = 20
k = 4

jsonFile = 'data/10k_artists_similarities.json'
G = createGraphFromJSON(jsonFile)


# READ or RE-READ the validation and train and test file
mode = 'r' #r for read, w for write only, a for append, r+ for r+w 
valF = open(validationFile,mode) 
trainF = open(trainFile,mode) 
testF = open(testFile,mode)



hits = 0
misses = 0

#Create user history objects
for user in range(0,maxNrUsers): 
    # READ next line, should be a user for both files
    valLine = valF.readline()
    trainLine = trainF.readline()
    testLine = testF.readline()
    
    #Deffensive: Check for empty files  
    if valLine == '' or trainLine == '' or testLine == '':
        print('End of file reached')
        break
    else:
        #We now know we have non-empty line        
        if (isUserLine(valLine) and isUserLine(trainLine) and isUserLine(testLine)):
            #We have users confirmed
            valUser = valLine.split('|')
            trainUser = trainLine.split('|')
            testUser = testLine.split('|')
            #userID's should match
            if(valUser[0] == trainUser[0] == testUser[0]):
                #Users matched
                userID = valUser[0]
                valRatings = int(valUser[1].rstrip())
                trainRatings = int(trainUser[1].rstrip())
                testRatings = int(testUser[1].rstrip())
                
                #print('Entered processing area')
                
                
                #Create user data from input files 
                userHistory = readUserData(trainRatings,trainF) 
                valData = readUserData(valRatings,valF)
                testData = readUserData(testRatings,testF)
                
                # NOTE: used to check statistics on the datasets
                #(h,m) = checkTrackExistence(G,userHistory[0])                
                #hits = hits + h
                #misses = misses + m
                
                # Turn user data into predictions
                print('Enters userID: %s'% (userID))
                print(predictOnGraph(G,valData,valRatings,userHistory,k))
                
                # Evaluate predictions
                
                #break
            else:
                print('Error: User ID mismatch')
                break
        else:
            print('ERROR: in file read process, user info line expected')
            break

#print('Hits: %d, Misses: %d' % (hits,misses))      

Enters userID: 0
([0, 0, 0, 0], 4)
Enters userID: 1
([0, 0, 0, 0], 4)
Enters userID: 2
([0, 0, 0, 0], 4)
Enters userID: 3
([0, 0, 0, 0], 3)
Enters userID: 4
([0, 70.835255078385671, 70.835255078385671, 0], 2)
Enters userID: 5
([0, 0, 0, 0], 4)
Enters userID: 6
([0, 0, 0, 0.073552628378269175], 3)
Enters userID: 7
([0, 0, 0, 0], 4)
Enters userID: 8
([0, 0, 0, 0], 4)
Enters userID: 9
([0, 0, 0, 0], 4)
Enters userID: 10
([0, 0, 0, 0], 3)
Enters userID: 11
([0, 0, 0, 0], 4)
Enters userID: 12
([0, 0, 0, 0], 4)
Enters userID: 13
([0, 0, 0, 0], 4)
Enters userID: 14
ERROR: too few elements in similarity list
ERROR: too few elements in similarity list
([0, 11.220593308736369, 11.220593308736369, 0], 2)
Enters userID: 15
([0, 0, 0, 0], 4)
Enters userID: 16
([0, 0, 0, 0], 4)
Enters userID: 17
([0, 0, 0, 0], 4)
Enters userID: 18
([46.271197586431803, 0, 46.520548949620242, 0], 2)
Enters userID: 19
([0, 0, 0, 0], 4)
Hits: 368, Misses: 1971


In [14]:

    
# Check if item track exists in userData object
def existsInData(track,userData):
    nrAttributes = len(userData)
    nrRatings = len(userData[0])
    
    for i in range(0,nrRatings):
        if track == userData[0][i]:
            return True
        
    #If we reach here, no match found    
    return False
        

In [29]:

#G.nodes()
#G.edges()

#trackID  
#            userData[0][i] = int(tabSplit[0])        
            #rating
#            userData[1][i] = int(tabSplit[1])
            #date 
#            userData[2][i] = int(tabSplit[2])
            #time 
#            userData[3][i] = tabSplit[3].rstrip()

In [109]:
#G.nodes()
#nodes[718]
#G.edges()

# G[20737] #In or out links shown? => use pred/succ instead
#G.neighbors(20737)
#G.nodes_with_selfloops()

#These are what we have For accessing nodes in relation to a specific node: 
#G.predecessors(20737)
#G.successors(20737)

#Accessing edges

#len(G)
#20737 in G

#list1 = []

#for i in G.neighbors(20737):
#    list1.append(i)

#len([]) == 0

#G[540429]

G[20737]

20737 in G

540429 in G


False

In [59]:
#keys = list(out.keys())
#len(keys)
#len(userHistory)
len(userHistory[0])

229

In [72]:
list2 = [1,2,3]

In [91]:
list2.extend([3])

In [88]:
list2

[1, 2, 3, 4, 4, None, [], 1]

In [110]:
valData

[[540429, 554898, 470288, 163101],
 [70, 90, 70, 70],
 [5112, 5112, 5118, 5229],
 ['11:24:00', '11:24:00', '19:58:00', '18:05:00']]

In [44]:
31569 in G.nodes()

True

In [39]:
G.nodes()

[524288,
 1,
 458756,
 342615,
 524300,
 196621,
 65557,
 65560,
 196633,
 458779,
 393244,
 31,
 65568,
 458787,
 262180,
 262182,
 131114,
 131120,
 152926,
 131134,
 327743,
 458816,
 393281,
 65610,
 196686,
 80,
 393298,
 196693,
 196695,
 65624,
 196697,
 65627,
 327772,
 65631,
 97,
 99,
 327781,
 327782,
 65642,
 589933,
 131183,
 600766,
 327801,
 393340,
 131203,
 327812,
 141,
 142,
 524431,
 131216,
 327704,
 393365,
 262296,
 393371,
 196767,
 65698,
 164,
 458917,
 65702,
 196776,
 360477,
 393392,
 590001,
 54643,
 590009,
 458938,
 196796,
 262333,
 131265,
 197,
 199,
 393416,
 393421,
 65742,
 458963,
 590038,
 524504,
 458788,
 393434,
 65756,
 262365,
 262367,
 131297,
 227,
 524519,
 327914,
 262379,
 131311,
 196848,
 524532,
 131317,
 131319,
 131323,
 131329,
 131330,
 260,
 524550,
 142039,
 524557,
 65809,
 524562,
 262419,
 327957,
 131355,
 290,
 327972,
 590117,
 524588,
 524590,
 65839,
 196915,
 262454,
 196922,
 393535,
 262464,
 131395,
 196933,
 622647